In [ ]:
'''
Stepper motor work by
On controller level input only step, direction, and timing

On driver level it's send pulse signal to the magnetic coils

Opened loop could miss step
We add angular encoder to the driver and correct this making it closed loop

on controller, it use PID control to check the position and recalculates the control signa until the target is reached.
'''

The following code provided by...

In [ ]:
import time

def pid_controller(kp, ki, kd, setpoint, process_value, dt, previous_error):
    """
    PID controller implementation.

    function pid_controller(kp, ki, kd, setpoint, process_value, dt, previous_error):
    calculate error as setpoint minus process_value
    calculate proportional term as kp times error
    calculate integral term as integral plus error times dt, then ki times integral
    calculate derivative term as (error minus previous_error) divided by dt, then kd times derivative
    calculate control output as proportional term plus integral term plus derivative term
    return control output and error

    Args:
        kp (float): Proportional gain.
        ki (float): Integral gain.
        kd (float): Derivative gain.
        setpoint (float): Desired value.
        process_value (float): Current process value.
        dt (float): Time step.
        previous_error (float): Previous error.

    Returns:
        float: Control output.
    """

    # Calculate error
    error = setpoint - process_value

    # Calculate proportional term
    p_term = kp * error

    # Calculate integral term
    integral = integral + error * dt
    i_term = ki * integral

    # Calculate derivative term
    derivative = (error - previous_error) / dt
    d_term = kd * derivative

    # Calculate control output
    control_output = p_term + i_term + d_term

    return control_output, error

In [ ]:
def auto_tune_pid(setpoint, process_value_function, dt, max_iterations=100, tolerance=0.01):
    """
    Auto-tunes PID controller using a Ziegler-Nichols method.
    
    function auto_tune_pid(setpoint, process_value_function, dt, max_iterations, tolerance):
    initialize PID gains to 0
    while iterations less than max_iterations:
        increase proportional gain until process oscillates
        determine ultimate gain (ku) and period (pu)
        calculate PID gains using Ziegler-Nichols method
        check for convergence
        if converged, break the loop
    return tuned PID gains

    Args:
        setpoint (float): Desired value.
        process_value_function (function): Function to get current process value.
        dt (float): Time step.
        max_iterations (int): Maximum number of iterations for auto-tuning.
        tolerance (float): Tolerance for convergence.

    Returns:
        tuple: Tuple of tuned PID gains (kp, ki, kd).
    """

    # Initialize PID gains
    kp = 0.0
    ki = 0.0
    kd = 0.0

    # Ziegler-Nichols method
    for iteration in range(max_iterations):
        # Increase proportional gain until process oscillates
        while True:
            kp += 0.1
            # Apply PID control and check for oscillation
            # ... (your oscillation detection logic here)
            if oscillation_detected:
                break

        # Determine ultimate gain and period
        ku = kp
        pu = 2 * pi / oscillation_frequency

        # Calculate PID gains
        kp = ku / 2
        ki = kp / (pu / 2)
        kd = kp * pu / 8

        # Check for convergence
        if abs(process_value_function() - setpoint) < tolerance:
            break

    return kp, ki, kd

In [ ]:
def step_motor_control(steps_per_revolution, target_degrees, target_speed, kp, ki, kd, dt):
    """
    Controls a step motor using PID control.

    Args:
        steps_per_revolution (int): Number of steps per revolution.
        target_degrees (float): Target rotation angle in degrees.
        target_speed (float): Target rotation speed in degrees/second.
        kp (float): Proportional gain for PID controller.
        ki (float): Integral gain for PID controller.
        kd (float): Derivative gain for PID controller.
        dt (float): Time step.

    Returns:
        None
    """

    # Calculate target steps and steps per second
    target_steps = int(steps_per_revolution * target_degrees / 360)
    target_steps_per_second = target_speed * steps_per_revolution / 360

    # Initialize PID controller
    previous_error = 0.0
    integral = 0.0

    # Step motor loop
    while True:
        # Get actual degree and speed from sensors
        actual_degrees = 1 # Replace with your sensor reading
        actual_speed = 1 # Replace with your sensor reading

        # Calculate error and speed error
        degrees_error = target_degrees - actual_degrees
        speed_error = target_speed - actual_speed

        # Calculate PID output
        pid_output = pid_controller(kp, ki, kd, target_steps_per_second, actual_speed, dt, previous_error)

        # Set step motor speed based on PID output
        # (Adjust step rate based on PID output)

        # Update PID controller variables
        previous_error = degrees_error
        integral += degrees_error * dt

        # Check if target reached
        if abs(degrees_error) < tolerance:
            break

        # Wait for dt
        time.sleep(dt)

# Example usage
if __name__ == "__main__":
    steps_per_revolution = 200
    target_degrees = 90.0
    target_speed = 30.0  # degrees/second
    kp = 1.0
    ki = 0.5
    kd = 0.2
    dt = 0.01  # Adjust time step as needed

    step_motor_control(steps_per_revolution, target_degrees, target_speed, kp, ki, kd, dt)

In [ ]:
def step_motor_control(steps_per_revolution, target_degrees, target_speed, kp, ki, kd, dt, microsteps_per_step):
    """
    Controls a step motor using PID control.

    Args:
        steps_per_revolution (int): Number of steps per revolution.
        target_degrees (float): Target rotation angle in degrees.
        target_speed (float): Target rotation speed in degrees/second.
        kp (float): Proportional gain for PID controller.
        ki (float): Integral gain for PID controller.
        kd (float): Derivative gain for PID controller.
        dt (float): Time step.
        microsteps_per_step (int): Number of microsteps per step.

    Returns:
        None
    """

    # Calculate target steps and steps per second
    target_steps = int(steps_per_revolution * target_degrees / 360)
    target_steps_per_second = target_speed * steps_per_revolution / 360

    # Initialize PID controller
    previous_error = 0.0
    integral = 0.0

    # Determine direction and set direction pin
    direction = 1 if target_steps >= 0 else -1
    set_direction_pin(direction)  # Replace with your function to set direction

    # Step motor loop
    while True:
        # Get actual degree and speed from sensors (replace with your implementation)
        actual_degrees = # ...
        actual_speed = # ...

        # Calculate error and speed error
        degrees_error = target_degrees - actual_degrees
        speed_error = target_speed - actual_speed

        # Calculate PID output
        pid_output = pid_controller(kp, ki, kd, target_steps_per_second, actual_speed, dt, previous_error)

        # Set step motor speed based on PID output
        step_rate = int(pid_output)  # Adjust step rate based on PID output
        delay_per_step = dt / step_rate

        # Step the motor
        for _ in range(microsteps_per_step):
            step_motor_step()  # Replace with your function to step the motor
            time.sleep(delay_per_step / microsteps_per_step)

        # Update PID controller variables
        previous_error = degrees_error
        integral += degrees_error * dt

        # Check if target reached
        if abs(degrees_error) < tolerance:
            break

# Example usage
if __name__ == "__main__":
    steps_per_revolution = 200
    target_degrees = 90.0
    target_speed = 30.0  # degrees/second
    kp = 1.0
    ki = 0.5
    kd = 0.2
    dt = 0.01  # Adjust time step as needed
    microsteps_per_step = 16

    step_motor_control(steps_per_revolution, target_degrees, target_speed, kp, ki, kd, dt, microsteps_per_step)

In [ ]:
# Example usage
if __name__ == "__main__":
    kp = 1.0
    ki = 0.5
    kd = 0.2
    setpoint = 100.0
    dt = 0.1
    previous_error = 0.0

    while True:
        process_value = # Get current process value from your system
        control_output, previous_error = pid_controller(
            kp, ki, kd, setpoint, process_value, dt, previous_error
        )

        # Apply control output to your system
        # ...

        time.sleep(dt)